In [2]:
import yaml

In [17]:
with open('words.json') as words_file:
    raw_words = yaml.load(words_file, yaml.CSafeLoader)

words = list(sorted(((x[0] if x is not None else {'word': w}) for (w, x) in raw_words.items()), key=lambda x: x['word'] if x is not None else '@'))

with open('words2.yaml', 'w') as words2_file:
    yaml.dump(words, words2_file)

In [21]:
with open('words2.yaml') as words_file:
    words = yaml.load(words_file, yaml.CSafeLoader)

import pandas as pd

rows = []
for word_response in words:
    word = word_response['word']
    if 'meanings' not in word_response:
        print(word, 'missing a definition')
    else:
        meanings = word_response['meanings']
        for meaning in meanings:
            partOfSpeech = meaning['partOfSpeech']
            for definition in (d['definition'] for d in meaning['definitions']):
                rows.append((partOfSpeech, word, definition))

pd.DataFrame(rows, columns=['partOfSpeech', 'word', 'definition'])

dovetail missing a definition
instransigent missing a definition
pr�cis missing a definition
repertorial missing a definition


,partOfSpeech,word,definition
0,verb,abase,"To lower, as in condition in life, office, ran..."
1,verb,abase,To lower physically; to depress; to cast or th...
2,verb,abase,"To lower in value, in particular by altering t..."
3,noun,abeyance,Expectancy; condition of ownership of real pro...
4,noun,abeyance,Suspension; temporary suppression; dormant con...
...,...,...,...
582,adjective,stentorian,"(of a voice) Loud, powerful, booming, suitable..."
583,adjective,stentorian,"(by extension) Stern, authoritarian; demanding..."
584,verb,stipulate,To require (something) as a condition of a con...
585,verb,stipulate,"To specify, promise or guarantee something in ..."


In [2]:
from word_database import *
definition_table = load_definition_table('words2.yaml')
part_of_speech_groups = definition_table.groupby('partOfSpeech')

dovetail missing a definition
instransigent missing a definition
pr�cis missing a definition
repertorial missing a definition


In [3]:
dir(part_of_speech_groups)
print(part_of_speech_groups.size())
print(part_of_speech_groups.get_group('adverb'))

partOfSpeech
adjective      220
adverb           3
noun           206
preposition      1
verb           157
dtype: int64
    partOfSpeech       word                                         definition
30        adverb      aloof  At or from a distance, but within view, or at ...
31        adverb      aloof                     Without sympathy; unfavorably.
228       adverb  erstwhile                             Formerly; in the past.


In [27]:
type(part_of_speech_groups.size())
import numpy as np
group_sizes = part_of_speech_groups.size()
parts_of_speech = group_sizes.index
parts_of_speech_probabilities = group_sizes / group_sizes.sum()
rng = np.random.default_rng()
selected_part_of_speech = rng.choice(group_sizes.index, 1, False, parts_of_speech_probabilities)[0]

alternative_definitions = part_of_speech_groups.get_group(selected_part_of_speech)
num_choices = min(4, alternative_definitions.size)
choice_index = rng.choice(alternative_definitions.index, num_choices, False)
choice_rows = alternative_definitions.loc[choice_index]
promoted_row_index = rng.choice(choice_rows.index, 1, False)
promoted_row = choice_rows.loc[promoted_row_index[0]]

import string
mcq = pd.DataFrame(choice_rows.index, index=list(string.ascii_uppercase[:num_choices]), columns=['choice_index'])
print(f'{promoted_row.word} ({promoted_row.partOfSpeech})')
for index, choice_row_index in mcq.iterrows():
    this_choice = choice_rows.loc[choice_row_index['choice_index']]
    print(f'{index}) {this_choice.definition}')


militate (verb)
A) To supply food to; to feast.
B) To give existence to, to produce (living creatures).
C) To give force or effect toward; to influence.
D) To defeat the plans or hopes of; to frustrate; disconcert.
